In [1]:
import pandas as pd
import numpy as np

# Load data input into df
df = pd.read_csv('sales_top10_skus.csv')

# Melt to long format: t, Year, Month stay as id_vars, others are SKUs
id_vars = ['t', 'Year', 'Month']
sku_columns = [col for col in df.columns if col not in id_vars]
df_long = df.melt(id_vars=id_vars, var_name='SKU', value_name='Sales')

# Remove possible $/comma formatting and convert to float
df_long['Sales'] = (df_long['Sales'].replace('[\$,]', '', regex=True)).astype(float)


In [2]:
# Add a truncated 'date' column

month_map = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}
df_long['Month_num'] = df_long['Month'].map(month_map)
df_long['Date'] = pd.to_datetime(df_long['Year'].astype(str) + '-' + df_long['Month_num'].astype(str) + '-01')


# Show the first 5 rows
df_long.head()

,t,Year,Month,SKU,Sales,Month_num,Date
0,1,2021,January,6HSD4J,811306.11,1,2021-01-01
1,2,2021,February,6HSD4J,660086.35,2,2021-02-01
2,3,2021,March,6HSD4J,849879.16,3,2021-03-01
3,4,2021,April,6HSD4J,1379693.06,4,2021-04-01
4,5,2021,May,6HSD4J,759690.80,5,2021-05-01


In [3]:
# Perform forecast for next 6 months using Holt's Exponential Smoothing

from statsmodels.tsa.holtwinters import ExponentialSmoothing
import pandas as pd

results = []
sku_list = df_long['SKU'].unique()

for sku in sku_list:
    sku_data = df_long[df_long['SKU'] == sku].sort_values('Date')
    y = sku_data['Sales'].values

    # Skip if too few points
    if pd.isnull(y).all() or (y == 0).all() or len(y) < 6:
        continue

    # Holt's linear trend model (trend='add', seasonal=None)
    model = ExponentialSmoothing(y, trend='add', seasonal=None)
    fit = model.fit(optimized=True)  # optimize both alpha and beta using statsmodels’ built-in optimization.
    forecast = fit.forecast(6)

    # Save the alpha and beta values
    best_alpha = fit.params['smoothing_level']
    best_beta = fit.params['smoothing_trend']
    
    # Forecast months
    last_date = sku_data['Date'].max()
    forecast_dates = pd.date_range(last_date + pd.offsets.MonthBegin(1), periods=6, freq='MS')
    for d, val in zip(forecast_dates, forecast):
        results.append({
            'SKU': sku,
            'Forecast_Month': d.strftime('%Y-%m'),
            'Forecasted_Sales': val
        })
    
print(">> Best Aplha: ", best_alpha)
print("\n>> Best Beta: ", best_beta, "\n")
        
forecast_df = pd.DataFrame(results)
print(forecast_df)


>> Best Aplha:  1.4901161193847656e-08

>> Best Beta:  2.9751444195078054e-10 

       SKU Forecast_Month  Forecasted_Sales
0   6HSD4J        2022-02      5.534532e+05
1   6HSD4J        2022-03      5.243769e+05
2   6HSD4J        2022-04      4.953005e+05
3   6HSD4J        2022-05      4.662242e+05
4   6HSD4J        2022-06      4.371479e+05
5   6HSD4J        2022-07      4.080716e+05
6   7XL27C        2022-02      9.083929e+04
7   7XL27C        2022-03      1.457403e+04
8   7XL27C        2022-04     -6.169124e+04
9   7XL27C        2022-05     -1.379565e+05
10  7XL27C        2022-06     -2.142218e+05
11  7XL27C        2022-07     -2.904870e+05
12  BJ30D6        2022-02      3.604623e+05
13  BJ30D6        2022-03      3.453712e+05
14  BJ30D6        2022-04      3.302801e+05
15  BJ30D6        2022-05      3.151890e+05
16  BJ30D6        2022-06      3.000979e+05
17  BJ30D6        2022-07      2.850068e+05
18   FJD6B        2022-02      1.724303e+06
19   FJD6B        2022-03      1.676907e

In [20]:
# Save output to csv file (long format for further analysis)
forecast_df.to_csv("forecast_next6m_long.csv", index=False)


In [22]:
# Optionally: Convert forecast_next6m.csv to wide format for easier read

# Pivot to wide format: each SKU as a column, rows are Forecast_Month
wide_df = forecast_df.pivot(index='Forecast_Month', columns='SKU', values='Forecasted_Sales').reset_index()

# To display in notebook without index
from IPython.display import display
display(wide_df.style.hide(axis='index')) 

# Save to CSV
wide_df.to_csv('forecast_next6m_wide.csv', index=False)

Forecast_Month,6HSD4J,7XL27C,BJ30D6,FJD6B,HK1R6J,LHR5LZ,NLDP86,SHZ5Y2,VJK56C,Y6HWKQ
2022-02,553453.177943,90839.290073,360462.340372,1724302.795660,245270.960750,208030.259070,188307.177688,289153.973728,1914772.010946,501754.773430
2022-03,524376.858390,14574.026265,345371.238919,1676906.568237,201941.413568,157563.241167,150292.156165,267823.017329,1729868.473156,504698.853712
2022-04,495300.538836,-61691.237544,330280.137467,1629510.340813,158611.866385,107096.223264,112277.134642,246492.060930,1544964.935367,507642.933995
2022-05,466224.219283,-137956.501352,315189.036014,1582114.113390,115282.319202,56629.205361,74262.113119,225161.104531,1360061.397578,510587.014277
2022-06,437147.899730,-214221.765161,300097.934562,1534717.885967,71952.772020,6162.187459,36247.091596,203830.148132,1175157.859788,513531.094559
2022-07,408071.580177,-290487.028969,285006.833109,1487321.658544,28623.224837,-44304.830444,-1767.929927,182499.191734,990254.321999,516475.174842
